<a href="https://colab.research.google.com/github/kshitijrajsharma/fAIr-utilities/blob/master/Package_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[fAIr](https://github.com/hotosm/fAIr): AI-assisted  Mapping Developer Setup

### fAIr is an open AI-assisted mapping service developed by the Humanitarian OpenStreetMap Team (HOT) that aims to improve the efficiency and accuracy of mapping efforts for humanitarian purposes. The service uses AI models, specifically computer vision techniques, to detect objects such as buildings, roads, waterways, and trees from satellite and UAV imagery.




Before you Continue Make sure you have docker container running with jupyter notebook and You connected your collab with your local runtime, Learn more https://github.com/hotosm/fAIr-utilities

## Setup fAIr Basemodel
1. Currently [ramp](https://rampml.global/) is supported  , Clone Ramp Scripts

In [ ]:
import os

if not os.path.exists('ramp-code'):
    !git clone https://github.com/kshitijrajsharma/ramp-code-fAIr.git ramp-code


Cloning into 'ramp-code'...
remote: Enumerating objects: 762, done.
remote: Counting objects: 100% (762/762), done.
remote: Compressing objects: 100% (572/572), done.
remote: Total 762 (delta 172), reused 750 (delta 163), pack-reused 0
Receiving objects: 100% (762/762), 32.86 MiB | 5.94 MiB/s, done.
Resolving deltas: 100% (172/172), done.


2. Download Ramp Basemodel checkpoint

In [ ]:
if not os.path.exists('ramp-code/checkpoint.tf'):
  !pip install gdown
  !gdown --fuzzy https://drive.google.com/file/d/1wvJhkiOrSlHmmvJ0avkAdu9sslFf5_I0/view?usp=sharing
  !unzip checkpoint.tf.zip -d ramp-code/ramp

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Downloading...
From (uriginal): https://drive.google.com/uc?id=1wvJhkiOrSlHmmvJ0avkAdu9sslFf5_I0
From (redirected): https://drive.google.com/uc?id=1wvJhkiOrSlHmmvJ0avkAdu9sslFf5_I0&confirm=t&uuid=d206ac41-c51b-4da3-b430-3ffbf8734b28
To: /tf/checkpoint.tf.zip
100%|████████████████████████████████████████| 113M/113M [00:16<00:00, 6.76MB/s]
Archive:  checkpoint.tf.zip
   creating: ramp-code/ramp/checkpoint.tf/
  inflating: ramp-code/ramp/__MACOSX/._checkpoint.tf  
  inflating: ramp-code/ramp/checkpoint.tf/.DS_Store  
  inflating: ramp-code/ramp/__MACOSX/checkpoint.tf/._.DS_Store  
  inflating: ramp-code/ramp/checkpoint.tf/keras_metadata.pb  
  inflating: ramp-code/ramp/checkpoint.tf/README.md  
   creating: ramp-code/ramp/checkpoint.tf/variables/
  inflating: ramp-code/ramp/checkpoint.tf/desktop.ini  
  inflating: ramp-code/ramp/checkpoint.tf/saved_model.pb  
  inflating: ramp-code/ramp/checkpoi

## Set your Env variable
Specify ramp home , Where you have it Installed , Remember ramp home is parent dir where ramp-code is

In [ ]:
import os
os.environ.update(os.environ)
# Add a new environment variable to the operating system for ramp home
os.environ["RAMP_HOME"] = os.path.join(os.getcwd())
print(os.environ["RAMP_HOME"])

/tf


# Prepare your Input Data

In [ ]:
import ramp.utils
import hot_fair_utilities
base_path = f"{os.getcwd()}/ramp-data/sample_2"

Segmentation Models: using `keras` framework.


## Preprocess input
Plain tile img 2 Georefrenced raster tif

In [ ]:
from hot_fair_utilities import preprocess
model_input_image_path = f"{base_path}/input"
preprocess_output=f"/{base_path}/preprocessed"
preprocess(
            input_path = model_input_image_path,
            output_path = preprocess_output,
            rasterize=True,
            rasterize_options=["binary"],
            georeference_images=True,
        )

Georeferencing for input: 100%|██████████| 147/147 [00:01<00:00, 73.55it/s]
/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
Clipping labels for input:   0%|          | 0/147 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a futu

# Train Model
Using preprocessed data run fine tuning or training itself

In [ ]:
from hot_fair_utilities import train
train_output = f"{base_path}/train"
final_accuracy, final_model_path = train(
    input_path=preprocess_output,
    output_path=train_output,
    epoch_size=2,
    batch_size=2,
    model="ramp",
    model_home=os.environ["RAMP_HOME"],
)

Starting to prepare data for training
Data is ready for training
Metric constructor function: get_sparse_categorical_accuracy_fn
Model: importing saved model /tf/ramp-code/ramp/checkpoint.tf
Starting Training with 2 epochs , 2 batch size , 62 steps per epoch , 11 validation steps......
Epoch 1/2
62/62 [==============================] - ETA: 0s - loss: 0.6998 - sparse_categorical_accuracy: 0.8185

INFO:tensorflow:Assets written to: /tf/ramp-data/sample_2/train/model-checkpts/20230621-071045/model_20230621-071045_001_0.862.tf/assets


INFO:tensorflow:Assets written to: /tf/ramp-data/sample_2/train/model-checkpts/20230621-071045/model_20230621-071045_001_0.862.tf/assets


62/62 [==============================] - 64s 719ms/step - loss: 0.6998 - sparse_categorical_accuracy: 0.8185 - val_loss: 0.3552 - val_sparse_categorical_accuracy: 0.8619
Epoch 2/2
62/62 [==============================] - ETA: 0s - loss: 0.3742 - sparse_categorical_accuracy: 0.8578

INFO:tensorflow:Assets written to: /tf/ramp-data/sample_2/train/model-checkpts/20230621-071045/model_20230621-071045_002_0.873.tf/assets


INFO:tensorflow:Assets written to: /tf/ramp-data/sample_2/train/model-checkpts/20230621-071045/model_20230621-071045_002_0.873.tf/assets


62/62 [==============================] - 40s 599ms/step - loss: 0.3742 - sparse_categorical_accuracy: 0.8578 - val_loss: 0.2700 - val_sparse_categorical_accuracy: 0.8727
Training Finished , Time taken to train : 104.51257558499856 seconds
Generating graphs ....
Graph generated at : /tf/ramp-data/sample_2/train/graphs
extracting highest accuracy model
/tf/ramp-data/sample_2/train/model-checkpts/20230621-071045
model_20230621-071045_002_0.873.tf


# Model Output : Checkpoints
Visulize your training accuracy and newly created training checkpoint

In [ ]:
print(final_accuracy,final_model_path)

87.3 /tf/ramp-data/sample_2/train/model-checkpts/20230621-071045/model_20230621-071045_002_0.873.tf


# Inference
Run predictions on newly created local model

In [ ]:
from hot_fair_utilities import predict
prediction_output = f"{base_path}/prediction/output"
predict(
    checkpoint_path=final_model_path,
    input_path=f"{base_path}/prediction/input",
    prediction_path=prediction_output,
)

Using : /tf/ramp-data/sample_2/train/model-checkpts/20230621-071045/model_20230621-071045_002_0.873.tf
It took 12 sec to load model
1/1 [==============================] - 1s 1s/step
It took 2 sec to predict with 0.5 Confidence Threshold


Georeferencing for output: 100%|██████████| 2/2 [00:00<00:00, 63.03it/s]

It took 0 sec to georeference


# Post process Predictions
Since inference will give the detected feature mask , Convert them to Geojson

In [ ]:
from hot_fair_utilities import polygonize
geojson_output= f"{prediction_output}/prediction.geojson"
polygonize(
    input_path=prediction_output,
    output_path=geojson_output,
    remove_inputs = True,
)

100%|██████████| 2/2 [00:00<00:00, 30.18mask/s]
/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
Merging components: 100%|██████████| 77/77 [00:14<00:00,  5.33component/s]
/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
